In [1]:
#define a new DFT which is unitary
#NOTE: in Beals' ['97] he normalizes by \sqrt{d_\lambda/n!}
#but also notes that a basis change is an equivalence relation on rep'ns
#and each equivalence class contains a unitary representatione
#if each rep'n \rho \in \hat{G} is unitary, then the transformation is unitary
#these representations are not unitary
#to make them unitary, use Weyl's unitary trick
#OPTION 1: use the formula P = \int_G \rho(g)\rho(g)^* dg, and take a square root to find Q s.t. P = Q^2
#OPTION 2: define a new invariant inner product compute an orthonormal basis w.r.t. 
#this inner product by computing the Gram matrix A and using A.gram_schmidt()

In [2]:
#compute a field which contains all the square roots required
def containing_field(SGA):
    required_square_roots = []
    for partition in Partitions(SGA.group().degree()):
        specht_module = SGA.specht_module(partition)
        rho = specht_module.representation_matrix
        group_size = SGA.group().cardinality()
        P = (1/group_size)*sum(rho(g)*rho(g).conjugate().transpose() for g in SGA.group())
        d, L = P.eigenmatrix_left()
        required_square_roots += [specht_module.dimension(),SGA.group().cardinality()] + d.diagonal()
    required_square_roots = flatten([[QQ(q).numerator(),QQ(q).denominator()] if q in QQ else q for q in required_square_roots])
    K = SGA.base_ring()
    for n in set(required_square_roots):
        R = PolynomialRing(K, 'x')
        x = R.gen()
        if (x**2 - n).is_irreducible():
            gen_name = "sqrt"+str(n).replace("/","over")
            K = K.extension(sqrt(n).minpoly(),names=gen_name)
    return K

In [3]:
#find the change-of-basis matrix Q making \rho(g) unitary for all g \in G
from sage.matrix.special import diagonal_matrix
from sage.misc.functional import sqrt
def unitary_change_of_basis(SGA,partition,K):
    rho = SGA.specht_module(partition).representation_matrix
    group_size = SGA.group().cardinality()
    P = (1/group_size)*sum(rho(g)*rho(g).conjugate().transpose() for g in SGA.group())
    d, L = P.eigenmatrix_left()
    return L.inverse() * diagonal_matrix([sqrt(K(a)) for a in d.diagonal()]) * L

In [4]:
#define the Fourier coefficient at the representation specht_module
#which is the Specht module corresponding to partition
def hat(f,partition,SGA,K,unitary=False):
    specht_module = SGA.specht_module(partition)
    rho = specht_module.representation_matrix
    if unitary:
        Q = unitary_change_of_basis(SGA,partition,K)
        unitary_factor = specht_module.dimension()/SGA.group().cardinality()
        sqrt_unitary_factor = sqrt(K(unitary_factor))
        return sqrt_unitary_factor*sum(f(g)*Q.inverse()*rho(g)*Q for g in SGA.group())
    else:
        return sum(f(g)*rho(g) for g in SGA.group())

In [5]:
#define the delta function delta_s(t) = {1 if s == t, 0 otherwise}
delta = lambda s: lambda t: 1 if t == s else 0

In [6]:
#for each basis element g \in G compute the Fourier coefficients \hat{\delta_g}(partition) for all partitions
from sage.misc.flatten import flatten
def unitary_dft(SGA):
    K = containing_field(SGA)
    return matrix(K,[flatten([hat(delta(g),partition,SGA,K,unitary=True).list() for partition in Partitions(SGA.group().degree())]) for g in SGA.group()]).transpose()

In [7]:
n = 3

In [8]:
SGA = SymmetricGroupAlgebra(QQbar,n)

In [9]:
K = containing_field(SGA); K

Algebraic Field

In [10]:
G = SGA.group()

In [11]:
SGA_dft = SGA.dft(); SGA_dft

[   1    1    1    1    1    1]
[   1  1/2   -1 -1/2 -1/2  1/2]
[   0  3/4    0  3/4 -3/4 -3/4]
[   0    1    0   -1    1   -1]
[   1 -1/2    1 -1/2 -1/2 -1/2]
[   1   -1   -1    1    1   -1]

In [12]:
#check if A*A^T == Id. it's not, but the columns are orthonormal
SGA_dft*SGA_dft.transpose()

[  6   0   0   0   0   0]
[  0   3   0   0   0   0]
[  0   0 9/4   0   0   0]
[  0   0   0   4   0   0]
[  0   0   0   0   3   0]
[  0   0   0   0   0   6]

In [13]:
partition = Partitions(SGA.group().degree())[1]; partition

[2, 1]

In [14]:
specht_module = SGA.specht_module(partition)

In [15]:
rho = specht_module.representation_matrix

In [16]:
group_size = SGA.group().cardinality()

In [17]:
P = (1/group_size)*sum(rho(g)*rho(g).conjugate().transpose() for g in SGA.group()); P.base_ring()

Algebraic Field

In [18]:
Q = unitary_change_of_basis(SGA,partition,K); Q

[ 1.115355071650411? 0.2988584907226845?]
[0.2988584907226845?  1.115355071650411?]

In [19]:
unitary_factor = specht_module.dimension()/SGA.group().cardinality(); unitary_factor

1/3

In [20]:
K

Algebraic Field

In [21]:
G[3]

[2, 3, 1]

In [22]:
U_rho = Q.inverse()*rho(G[3])*Q; U_rho

[-0.500000000000000?  0.866025403784439?]
[-0.866025403784439? -0.500000000000000?]

In [23]:
U_rho*U_rho.conjugate().transpose()

[1.000000000000000?            0.?e-17]
[           0.?e-17 1.000000000000000?]

In [24]:
unitary_dft(SGA)

[ 0.4082482904638630?  0.4082482904638630?  0.4082482904638630?  0.4082482904638630?  0.4082482904638630?  0.4082482904638630?]
[ 0.5773502691896258?                    0   0.500000000000000? -0.2886751345948129? -0.2886751345948129?  -0.500000000000000?]
[             0.?e-18  0.5773502691896258? -0.2886751345948129?   0.500000000000000?  -0.500000000000000? -0.2886751345948129?]
[             0.?e-18  0.5773502691896258? -0.2886751345948129?  -0.500000000000000?   0.500000000000000? -0.2886751345948129?]
[ 0.5773502691896258?                    0  -0.500000000000000? -0.2886751345948129? -0.2886751345948129?   0.500000000000000?]
[ 0.4082482904638630? -0.4082482904638630? -0.4082482904638630?  0.4082482904638630?  0.4082482904638630? -0.4082482904638630?]

In [25]:
U_dft = unitary_dft(SGA); U_dft*U_dft.conjugate().transpose()

[1.000000000000000?            0.?e-18            0.?e-18            0.?e-18            0.?e-18            0.?e-18]
[           0.?e-18 1.000000000000000?            0.?e-18            0.?e-18            0.?e-18            0.?e-18]
[           0.?e-18            0.?e-18 1.000000000000000?            0.?e-18            0.?e-18            0.?e-18]
[           0.?e-18            0.?e-18            0.?e-18 1.000000000000000?            0.?e-18            0.?e-18]
[           0.?e-18            0.?e-18            0.?e-18            0.?e-18 1.000000000000000?            0.?e-18]
[           0.?e-18            0.?e-18            0.?e-18            0.?e-18            0.?e-18 1.000000000000000?]

In [26]:
#check that the DFT is unitary
(U_dft*U_dft.transpose()) == identity_matrix(SGA.group().cardinality())

True

In [27]:
#QUESTION: what are the eigenvalues?
#need to be careful as characteristic polynomial is degree n! over K, so [L:QQ] = [L:K]*[K:QQ] = n!*[K:QQ]
#for n=3, [L:QQ] = n!*[K:QQ] = 6*4 = 24

In [28]:
U_dft.charpoly()

x^6 + 1.0773502691897?*x^5 - 0.924311009994?*x^4 - 2.10095363131?*x^3 - 0.92431101000?*x^2 + 1.077350269190?*x + 1.00000000000?

In [29]:
#ISSUE: the splitting field appears to be high degree, and the coefficients are large
#the defining polynomial is monic with integer coefficients
#since the eigenvalues lie on the unit circle, a result of Kroenecker implies they are roots of unity
if len(list(U_dft.charpoly().factor())) != U_dft.charpoly().degree():
    L.<a> = U_dft.charpoly().splitting_field(); L
else:
    L = K

In [30]:
L

Algebraic Field

In [31]:
#one can factor the polynomial over a splitting field L/K
#but there is no way to express the roots of a quintic in terms of radicals
eigs = matrix(L,U_dft).eigenvalues(extend=False); eigs

[-0.9865699009578278? - 0.1633396171296542?*I, -0.9865699009578278? + 0.1633396171296542?*I, -0.5437444089082021? - 0.8392508670124029?*I, -0.5437444089082021? + 0.8392508670124029?*I, 0.9916391752712170? - 0.1290416447020912?*I, 0.9916391752712170? + 0.1290416447020912?*I]

In [55]:
phi_inverse = [n for n in range(1000) if euler_phi(n) == 24]; phi_inverse
for n in phi_inverse:
    for k in range(n):
        print(exp(2*pi*I*k/n).n(100) - eigs[0].n(100))

1.9865699009578277220284736329 + 0.16333961712965412484112107495*I
1.9704994895564573774241097269 + 0.34189651192829077285483944254*I
1.9228047715975649315372293574 + 0.51471444121099682972844434546*I
1.8450186945596938405541645883 + 0.67623889453556026874961472789*I
1.7396413669614386555613717456 + 0.82127834306936673721124025695*I
1.6100597028165612525534785169 + 0.94517109959768393354956560163*I
1.4604385634308263927368566426 + 1.0439351489863921200340311371*I
1.2955868953327751461307670501 + 1.1143961334248076969575604083*I
1.1208031667754831980654922744 + 1.1542893788975888800898077881*I
0.94170507060731279657038329612 + 1.1623326836709687722131769834*I
0.76404896700151331773957106842 + 1.1382675293114777318592527579*I
0.59354486930390410384050612310 + 1.0828673896811047631630976657*I
0.43567291950572519934136931380 + 0.99791287085095677577433175183*I
0.29550725197096304612353820726 + 0.88613448095704565338638442645*I
0.17755290658288029792618021573 + 0.751124869422127254009827029

In [59]:
eigs[0].minpoly()

x^24 + 2*x^23 - 1/2*x^22 - 17/6*x^21 - 215/144*x^20 - 1/6*x^19 + 509/216*x^18 + 11/3*x^17 - 35/1296*x^16 - 139/54*x^15 - 1483/648*x^14 + 7/18*x^13 + 155/54*x^12 + 7/18*x^11 - 1483/648*x^10 - 139/54*x^9 - 35/1296*x^8 + 11/3*x^7 + 509/216*x^6 - 1/6*x^5 - 215/144*x^4 - 17/6*x^3 - 1/2*x^2 + 2*x + 1

In [34]:
rational_approx_args = [(arg(eig)/(2*pi)).n(100).nearby_rational(max_denominator=100_000_000_000_000_000_000_000) for eig in eigs]; rational_approx_args

[-10782678805850241855303/22753706999878039149494,
 10782678805850241855303/22753706999878039149494,
 -19751357844255063647819/57837581692632446111637,
 19751357844255063647819/57837581692632446111637,
 -229510577551663428287/11143970750595710312210,
 229510577551663428287/11143970750595710312210]

In [35]:
#galois_group() appears to be the absolute version
#NOTE: for a splitting field L, we should have [L:K] = |Gal(L/K)|
if L != L.algebraic_closure():
    L_G = L.galois_group()

In [36]:
#n=3: two real, two complex
#n=4: all complex
#the magnitude is not 1, they're closely grouped around 2 or 3

In [37]:
SGA_eigs = SGA_dft.eigenvalues(); SGA_eigs

[2.203130118946493?, 0.6196927916815408? + 1.649489291766649?*I, 0.6196927916815408? - 1.649489291766649?*I, -1.180831595337157? + 1.549010082939973?*I, -1.180831595337157? - 1.549010082939973?*I, -2.080852511635262?]

In [38]:
[abs(eig) for eig in SGA_eigs]

[2.203130118946493?,
 1.762053937799550?,
 1.762053937799550?,
 1.947766796512405?,
 1.947766796512405?,
 2.080852511635262?]

In [39]:
#note that the singluar values are the square roots of the diagonal entries of the Gram matrix
print(SymmetricGroup(n).algebra(CDF).dft().SVD()[1].numpy().diagonal())
print(sqrt((SGA_dft*SGA_dft.transpose()).numpy().diagonal()))

[2.44948974+0.j 2.44948974+0.j 2.        +0.j 1.73205081+0.j
 1.73205081+0.j 1.5       +0.j]
[2.449489742783178? 1.732050807568878? 3/2 2 1.732050807568878?
 2.449489742783178?]
